# Example of how to perform parrallel calculations with DiKErnel

This tutorial shows how to setup, run and analyse the speed of a DiKErnel calculation using pydrever. It deals with:
1. [Specifying calculation input](#calculation-input)
2. [Create a calculator object](#create-calculator)
3. [Measure calculation performance](#measure-performance)
4. [Switch between parallel and serial calculations](#switch-parallel)


<div class="alert alert-block alert-info">
<b>Not necessary:</b> The following code makes sure that also during development pydrever can be adressed properly.
</div>

In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

# Calculation input <a id="calculation-input"></a>
This is an example

In [2]:
import pydrever.data as data

x_positions = [0.0, 25.0, 35.0, 41.0, 45, 50, 60, 70]
z_positions = [-3, 0.0, 1.5, 1.7, 3.0, 3.1, 0, -1]
roughnesses = [1, 1, 0.75, 0.5, 0.8, 0.8, 0.8]
dike_schematization = data.DikeSchematization(
    dike_orientation=90.0, 
    x_positions=x_positions, 
    z_positions=z_positions, 
    roughnesses=roughnesses, 
    x_outer_toe=25.0, 
    x_outer_crest=45.0,
    foreshore_slope=0.05,
    z_bottom=-3
)

In [3]:
import numpy
from pydrever.data import HydrodynamicConditions

def create_hydrodynamic_conditions(n_steps:int = 100) -> HydrodynamicConditions:
    time_steps = numpy.linspace(
        0.0, 126000.0, int(126000.0 / n_steps), dtype=float, endpoint=True
    )

    phase_water_levels = numpy.pi / 64.0
    amplitude_water_levels = 1.3
    minimum_water_level = 0.5
    water_levels = (
        amplitude_water_levels
        * (1 - numpy.cos(time_steps[1:] / 1000.0 * phase_water_levels))
        + minimum_water_level
    )

    phase_wave_heights = numpy.pi / 32.0
    ampltude_waves = 0.2
    minimum_wave_height = 0.8
    wave_heights = (
        ampltude_waves * (1 - numpy.cos(time_steps[1:] / 1000.0 * phase_wave_heights))
        + minimum_wave_height
    )

    wave_periods = 7.0 + time_steps[1:] / 1000.0 * 0.05
    
    wave_directions = 10 + time_steps[1:] * 0.0

    hydrodynamic_conditions = HydrodynamicConditions(
        time_steps=time_steps.tolist(), 
        water_levels=water_levels.tolist(), 
        wave_heights=wave_heights.tolist(), 
        wave_periods=wave_periods.tolist(), 
        wave_directions=wave_directions.tolist()
    )

    return hydrodynamic_conditions

hydrodynamic_conditions = create_hydrodynamic_conditions(100)

In [4]:
grass_wave_runup_layer = data.GrassWaveRunupLayerSpecification(
    outer_slope=0.3, 
    top_layer_type=data.TopLayerType.GrassClosedSod)

def create_revetment_zone(nx:int=5):
    return data.RevetmentZoneSpecification(
        zone_definition=data.HorizontalRevetmentZoneDefinition(x_min=41.1, x_max=44.9, nx=nx), 
        top_layer_specification=grass_wave_runup_layer),

revetmen_zone = create_revetment_zone(20)

In [5]:
def create_calculation_input(
        nx:int=10,
        n_times:int=100
        ):
    input = data.DikernelInput(
        dike_schematization=dike_schematization, 
        hydrodynamic_input=create_hydrodynamic_conditions(n_times)
    )

    input.output_revetment_zones = list(create_revetment_zone(nx))

    return input

# Create calculator <a id="create-calculator"></a>

In [6]:
from pydrever.calculation import Dikernel

dikernel = Dikernel(create_calculation_input())

dikernel.calculate_locations_parallel = True
dikernel.calculate_time_steps_parallel = True

# Measure calculator performance <a id="measure-performance"></a>

In [7]:
import time

def perform_timed_calculation(
        nx:int = 5,
        n_times:int = 100,
        times_parallel = False,
        locations_parallel = False
):

    input = create_calculation_input(nx=nx, n_times=n_times)
    dikernel = Dikernel(input)
    dikernel.calculate_time_steps_parallel = times_parallel
    dikernel.calculate_time_steps_parallel = locations_parallel

    start_time = time.time()

    dikernel.run()

    end_time = time.time()
    return end_time - start_time



# Switch between parallel and serial calculations <a id="switch-parallel"></a>

In [8]:
print(perform_timed_calculation(n_times=10, nx=100))
print(perform_timed_calculation(n_times=100, nx=100))
print(perform_timed_calculation(n_times=1000, nx=100))
print(perform_timed_calculation(n_times=10000, nx=100))
print(perform_timed_calculation(n_times=100000, nx=100))

print(perform_timed_calculation(n_times=10, nx=100, times_parallel=True))
print(perform_timed_calculation(n_times=100, nx=100, times_parallel=True))
print(perform_timed_calculation(n_times=1000, nx=100, times_parallel=True))
print(perform_timed_calculation(n_times=10000, nx=100, times_parallel=True))
print(perform_timed_calculation(n_times=100000, nx=100, times_parallel=True))

print(perform_timed_calculation(n_times=10, nx=100, times_parallel=True, locations_parallel=True))
print(perform_timed_calculation(n_times=100, nx=100, times_parallel=True, locations_parallel=True))
print(perform_timed_calculation(n_times=1000, nx=100, times_parallel=True, locations_parallel=True))
print(perform_timed_calculation(n_times=10000, nx=100, times_parallel=True, locations_parallel=True))
print(perform_timed_calculation(n_times=100000, nx=100, times_parallel=True, locations_parallel=True))

18.34024167060852
1.817399501800537
0.18733906745910645
0.014938116073608398
0.0
17.863547801971436
1.7996349334716797
0.1892256736755371
0.027189016342163086
0.0
43.477911710739136
3.825951337814331
0.31696557998657227
0.03891158103942871
0.0
